In [228]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [229]:
df = pd.read_csv('winequality-red.csv')

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [230]:
X = df.drop(['quality'],axis = 1)
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [231]:
df.loc[df['quality'] == 3,'quality'] = 1
df.loc[df['quality'] == 4,'quality'] = 2
df.loc[df['quality'] == 5,'quality'] = 3
df.loc[df['quality'] == 6,'quality'] = 4
df.loc[df['quality'] == 7,'quality'] = 5
df.loc[df['quality'] == 8,'quality'] = 6

y = df['quality']
y.head()


0    3
1    3
2    3
3    4
4    3
Name: quality, dtype: int64

In [232]:
X.shape, y.shape

((1599, 11), (1599,))

In [233]:
from sklearn.preprocessing import OneHotEncoder  
encoder = OneHotEncoder(sparse = False)
y = y[:,np.newaxis]
y_onehot = encoder.fit_transform(y)
y_onehot.shape



(1599, 6)

In [234]:
#Show coordinate mapping
y[0], y_onehot[0,:]

(array([3], dtype=int64), array([ 0.,  0.,  1.,  0.,  0.,  0.]))

In [235]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def forward_propogate(X,theta1,theta2):
    m = X.shape[0]
    a1 = np.insert(X,0,values=np.ones(m),axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2),0,values=np.ones(m), axis=1)
    z3 = a2*theta2.T
    h = sigmoid(z3)
    
    return a1,z2,a2,z3,h

def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):  
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propogate(X, theta1, theta2)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))  
    return J

In [236]:
#Setup

input_size = 11
hidden_size = 8
num_labels = 6
learning_rate = 1.3
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((8, 12), (6, 9))

In [237]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)


4.2932632793132051

In [238]:
def sigmoid_gradient(z):  
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [239]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):  

    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propogate(X, theta1, theta2)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    ##### end of cost function logic, below is the new part #####

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d3t = ht - yt  # (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    delta1 = delta1 / m
    delta2 = delta2 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [240]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)  
J, grad.shape


(4.2932632793132051, (150,))

In [246]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),  
                method='TNC', jac=True, options={'maxiter': 250})
fmin  

C:\Users\Nishant\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


     fun: 1.677648985754759
     jac: array([  1.94027426e-04,   8.68354376e-04,  -1.58279271e-04,
         6.51660716e-05,   2.00020196e-03,   1.45446393e-04,
         5.94991807e-04,   1.10159407e-02,   1.76987581e-04,
         5.91948256e-04,   6.43884140e-04,   3.26665720e-03,
        -1.59108542e-05,  -7.69165532e-04,  -2.05957574e-04,
        -4.21592799e-05,  -1.65300892e-04,  -7.03409918e-06,
         2.33495512e-03,   1.10928024e-03,  -9.13476195e-06,
         1.56815075e-05,   5.50156195e-05,  -8.78159197e-05,
        -5.82331212e-04,   1.76148583e-03,  -7.10934431e-04,
        -6.28195255e-04,  -1.19859040e-02,  -3.98476267e-04,
        -3.22906958e-03,  -1.65595737e-01,  -1.32150069e-03,
        -6.85035965e-03,  -1.59997655e-03,  -3.60740465e-02,
        -4.22601373e-04,  -3.08003094e-03,  -2.77133988e-04,
        -3.03373357e-05,  -1.45522358e-03,  -2.50720321e-05,
        -1.40773014e-03,  -5.90776242e-03,  -4.39822222e-04,
        -1.53265284e-03,  -3.33137704e-04,  -4.

In [247]:
X = np.matrix(X)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propogate(X, theta1, theta2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  
y_pred 

array([[3],
       [3],
       [3],
       ..., 
       [4],
       [3],
       [4]], dtype=int64)

In [248]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print ('accuracy = {0}%'.format(accuracy * 100))


accuracy = 60.47529706066291%


In [249]:
y

matrix([[3],
        [3],
        [3],
        ..., 
        [4],
        [3],
        [4]], dtype=int64)